## Importing Data and Libraries

In [3]:
import os
import numpy as np
import string
from tqdm.notebook import tqdm
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential, callbacks, utils

In [4]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

## Preprocessing

In [ ]:
def tokenize(data, to_select_lines=-1):

  result = []
  table = str.maketrans('', '', string.punctuation)

  idx = 0

  for line in data:
    line_tokens = []
    idx += 1
    if (to_select_lines == -1):
      idx -= 1
    if (idx == to_select_lines):
      break

    for word in line.split():
      if len(word.translate(table)) > 1:
        line_tokens.append(word.translate(table))

    result.append(line_tokens)

  return result

In [ ]:
def create_dict(tokens, to_select_words=-1):

  word_dict = {}
  for i in tokens:
    for j in i:
      try:
        word_dict[j] += 1
      except:
        word_dict[j] = 1

  res_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}


  if (to_select_words == -1):
    final_dict = {}
    for idx, (key, value) in zip(range(len(res_dict)), res_dict.items()):
      final_dict[key] = idx
    return final_dict

  final_dict = {}
  for idx, (key, value) in zip(range(to_select_words), res_dict.items()):
    final_dict[key] = idx

  return final_dict

In [ ]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)

  return result

In [ ]:
def get_sequences(data, word_dict, n_gram_length):

  input_sequence = []
  idx = 0

  for line in data:
    token_list = convert(line, word_dict)
    for i in range(1, len(token_list)):
      n_gram_seq = token_list[i:i+n_gram_length+1]
      if (len(n_gram_seq) == n_gram_length):
        input_sequence.append(n_gram_seq)

  return input_sequence

In [ ]:
def preprocess(data, to_select_lines=-1, to_select_words=-1, n_gram_length=3):

  print("Tokenizing")
  tokenized = tokenize(data, to_select_lines)
  print("Creating Dictionary")
  word_dict = create_dict(tokenized, to_select_words)
  print("Creating Input Sequences")
  input_sequences = get_sequences(tokenized, word_dict, n_gram_length)

  del tokenized

  return word_dict, input_sequences

## Changing data for input

In [ ]:
word_dict, input_sequences = preprocess(data, to_select_words=10000, n_gram_length=3)

Tokenizing
Creating Dictionary
Creating Input Sequences


In [ ]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [ ]:
len(input_sequences), len(word_dict)

(97614, 10000)

In [ ]:
input_sequences = np.array(input_sequences)
np.random.shuffle(input_sequences)
input_sequences[:3]

array([[  13,   69,   41],
       [  17,    0,    9],
       [  15, 1519,    9]])

In [ ]:
def data_processor(input_sequences, word_dict):

  x = input_sequences[:, :-1]
  label = input_sequences[:, -1]

  temp_array = np.zeros((label.shape[0], len(word_dict)))

  idx=0
  for i in label:
    temp_array[idx][i] = 1
    idx += 1

  return x, temp_array

## Saving the dictionary and sequence

In [ ]:
a = pd.DataFrame(data = word_dict.keys())
a['values'] = word_dict.values()
a.columns = ['words', 'values']
a.head()
a.to_csv('/content/drive/MyDrive/Next word Generation/Dictionary/word_dict_1.csv', index=False)

In [ ]:
np.save('/content/drive/MyDrive/Next word Generation/Sequences/seq_1.npy', input_sequences)

## Creating Model

In [ ]:
def create_model(word_dict):

  model = Sequential([
                    layers.Embedding(len(word_dict), 256),
                    layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
                    layers.Bidirectional(layers.LSTM(256)),
                    layers.Dense(1024, activation='relu'),
                    layers.Dense(len(word_dict), activation='softmax')
          ])
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

## Training the model

In [ ]:
def check_results(input_string, model, word_dict, reverse_dict, generation_len=10):

  test_input = input_string
  next_words = generation_len
  table = str.maketrans('', '', string.punctuation)

  for i in range(next_words):
    token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
    token_list = convert(token_list, word_dict)
    token_list = [token_list[-2:]]
    predicted =  model.predict_classes(token_list, verbose=0)[0]
    
    output_word = ''
    if predicted in reverse_dict:
      output_word = reverse_dict[predicted]
    test_input += " " + output_word

    print(test_input)

In [ ]:
model = create_model(word_dict)
start = 0

for end in range(10001, len(input_sequences), 30000):

  print("\n\n############# Iteration Start #############\n\n")

  x, y = data_processor(input_sequences[start: end], word_dict)

  save_point = callbacks.ModelCheckpoint('/content/drive/MyDrive/Next word Generation/Models/M1.h5', monitor='loss', save_best_only=False)
  stop_check = callbacks.EarlyStopping(
                  monitor='loss', min_delta=0, patience=4, verbose=0,
                  mode='auto', baseline=None, restore_best_weights=False
                )

  model.fit(x, y, epochs=20, callbacks=[save_point, stop_check], batch_size=256)

  start = end

  print("\n\n Checking Resuls \n\n")

  check_results('ਮੇਰਾ ਭਾਰਤ', model, word_dict, reverse_dict)

  print("\n\n############# Iteration End #############\n\n")



############# Iteration Start #############


Epoch 1/20
40/40 [==============================] - 6s 41ms/step - loss: 7.8338 - accuracy: 0.2027
Epoch 2/20
40/40 [==============================] - 2s 41ms/step - loss: 3.7615 - accuracy: 0.2292
Epoch 3/20
40/40 [==============================] - 2s 43ms/step - loss: 3.4893 - accuracy: 0.2402
Epoch 4/20
40/40 [==============================] - 2s 41ms/step - loss: 3.4495 - accuracy: 0.2384
Epoch 5/20
40/40 [==============================] - 2s 42ms/step - loss: 3.4296 - accuracy: 0.2384
Epoch 6/20
40/40 [==============================] - 2s 40ms/step - loss: 3.3165 - accuracy: 0.2588
Epoch 7/20
40/40 [==============================] - 2s 41ms/step - loss: 3.1104 - accuracy: 0.3152
Epoch 8/20
40/40 [==============================] - 2s 41ms/step - loss: 2.8362 - accuracy: 0.3650
Epoch 9/20
40/40 [==============================] - 2s 41ms/step - loss: 2.5286 - accuracy: 0.4256
Epoch 10/20
40/40 [==============================] - 2s 40ms/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ਮੇਰਾ ਭਾਰਤ ਪਏ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ।
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ ਭਾਰੀ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ ਭਾਰੀ ਹੈ।
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ ਭਾਰੀ ਹੈ। ਦੇ
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ ਭਾਰੀ ਹੈ। ਦੇ ਰਹੇ।
ਮੇਰਾ ਭਾਰਤ ਪਏ ਹਨ ਹੋਈ ਹੈ। ਰੁਝਾਨਰਿਹਾ ਭਾਰੀ ਹੈ। ਦੇ ਰਹੇ। ਦੇ


############# Iteration End #############




############# Iteration Start #############


Epoch 1/20
118/118 [==============================] - 5s 42ms/step - loss: 2.9518 - accuracy: 0.4008
Epoch 2/20
118/118 [==============================] - 5s 41ms/step - loss: 2.3313 - accuracy: 0.4698
Epoch 3/20
118/118 [==============================] - 5s 41ms/step - loss: 2.0608 - accuracy: 0.5028
Epoch 4/20
118/118 [==============================] - 5s 41ms/step - loss: 1.8680 - accuracy: 0.5342
Epoch 5/20
118/118 [==============================] - 5s 40ms/step - loss: 1.7135 - accuracy: 0.5590
Epoch 6/20
118/118 [==============================] - 5s 40ms

## Visualizing the results

In [ ]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

In [ ]:
data[50:55]

["ਇਸ ਤੋਂ ਪਹਿਲਾਂ ਸਨਿਚਰਵਾਰ ਭਾਰਤ ਸਰਕਾਰ ਨੇ ਕਿਹਾ ਕਿ ਉਹ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕਾਂ ਵਲੋਂ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਨੂੰ ਮਾਰ ਕੇ ਉਨ੍ਹਾਂ ਦਾ ਸਿਰ ਕਲਮ ਕਰਕੇ ਲਿਜਾਣ ਵਿਰੁੱਧ ਬਦਲੇ ਦੀ ਕਰਵਾਈ ਨਹੀਂ ਕਰੇਗਾ ਪਰ ਏਅਰ ਚੀਫ ਮਾਰਸ਼ਲ ਨਾਰਮਨ ਅਨਿਲ ਕੁਮਾਰ ਬਰਾਉਨ ਨੇ ਕਲ੍ਹ ਕਿਹਾ ਸੀ ਕਿ 2003 ਤੋਂ ਕੰਟਰੋਲ ਰੇਖਾ 'ਤੇ ਚਲੀ ਆ ਰਹੀ ਜੰਗਬੰਦੀ ਦੀ ਪਾਕਿ ਵਲੋਂ ਉਲੰਘਣਾ ਕਦੇ ਵੀ ਬਰਦਾਸ਼ਤ ਨਹੀਂ ਕੀਤੀ ਜਵੇਗੀ |\n",
 "ਅਸੀਂ ਸਥਿਤੀ 'ਤੇ ਸਾਵਧਾਨੀ ਨਾਲ ਨਜ਼ਰ ਰੱਖ ਰਹੇ ਹਾਂ |\n",
 'ਜੇਕਰ ਉਲੰਘਣਾ ਜਾਰੀ ਰਾਹੀ ਤਾਂ ਅਸੀਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਸੋਚਾਂਗੇ ਪਰ ਉਨ੍ਹਾਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਦੱਸਣ ਤੋਂ ਇਨਕਾਰ ਕਰ ਦਿੱਤਾ ਸੀ |\n',
 "ਜੰਮੂ ਤੇ ਕਸ਼ਮੀਰ ਵਿਚ ਭਾਰਤੀ ਫ਼ੌਜ ਵਲੋਂ ਕਥਿਤ ਰੂਪ ਵਿਚ ਕੀਤੀ ਗੋਲੀਬਾਰੀ ਵਿਚ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕ ਦੇ ਮਾਰੇ ਜਣ ਪਿੱਛੋਂ 6 ਜਨਵਰੀ ਤੋਂ ਹੀ ਭਾਰਤ ਅਤੇ ਪਾਕਿ ਵਿਚਕਾਰ ਸਰਹੱਦ 'ਤੇ ਤਣਾਅ ਬਣਿਆਂ ਹੋਇਆ ਹੈ |\n",
 'ਦੋ ਦਿਨ ਪਿੱਛੋਂ ਪਾਕਿ ਸੈਨਿਕ ਪੁਣਛ ਜ਼ਿਲ੍ਹੇ ਵਿਚ ਮੇਂਧਰ ਸੈਕਟਰ ਵਿਚ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਦਾ ਸਿਰ ਵੱਢ ਕੇ ਲੈ ਗਏ |\n']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Next word Generation/Dictionary/word_dict_1.csv')
model = tf.keras.models.load_model('/content/drive/MyDrive/Next word Generation/Models/M1.h5')

In [ ]:
word_dict = {}
for i,j in zip(df['words'], df['values']):
  word_dict[i] = j

In [ ]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [ ]:
table = str.maketrans('', '', string.punctuation)

In [ ]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)
  return result

In [ ]:
test_input = 'ਉਹ ਕਰੇਗਾ'
next_words = 10
array = np.zeros((10, len(word_dict)))

for i in range(next_words):
  token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
  token_list = convert(token_list, word_dict)
  token_list = [token_list[-2:]]
  array[i] = model.predict(token_list, verbose=0)[0]
  predicted =  model.predict_classes(token_list, verbose=0)[0]
  output_word = ''
  if predicted in reverse_dict:
    output_word = reverse_dict[predicted]
  test_input += " " + output_word

  print(test_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ਉਹ ਕਰੇਗਾ ਦੇ
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ।
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ।
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ ਸੀ।
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ ਸੀ। ਹੈ।
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ ਸੀ। ਹੈ। ਮਿਲੀ
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ ਸੀ। ਹੈ। ਮਿਲੀ ਸੀ।
ਉਹ ਕਰੇਗਾ ਦੇ ਗਏ। ਦੇ ਹੈ। ਦੇ ਸੀ। ਹੈ। ਮਿਲੀ ਸੀ। ਕੀਰਤਨ
